# Comment Model (Model 1 in pipeline)

This model will generate a comment based on a user-inputed topic.

### I. Load Data

In [ ]:
!pip install -U accelerate --quiet
!pip install transformers --quiet

# Mount your Google Drive to access the data
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change working directory to your folder path
%cd drive/MyDrive/AI Camp/Captivating Cupcakes/data
#%cd /content/drive/MyDrive/Captivating Cupcakes/data

%pwd

/content/drive/MyDrive/AI Camp/Captivating Cupcakes/data


'/content/drive/MyDrive/AI Camp/Captivating Cupcakes/data'

In [ ]:
import pandas as pd
results = pd.read_csv('comments_results.csv') #list of true/false
results = results.iloc[:, 0].tolist()

In [ ]:
rc = pd.read_csv('reddit_comments.csv')

rc_negative = rc.query('score < 0')

In [ ]:
rc_negative.head()

,subreddit,body,controversiality,score
20,leagueoflegends,"TLDR:\n\n""We invested a lot of brain power and...",0,-4
23,nba,[the only reason anyone knows who Jared Dudley...,0,-5
30,news,This is disgusting every week,0,-1
31,nba,So you’d say 26 year old Buddy Hield isn’t a p...,1,-2
49,AskReddit,Not in my findings,1,-1


In [ ]:
rc = rc_negative[[False if value else True for value in filtered_rc]].reset_index(drop=True)

In [ ]:
rc_data = []
for idx, row in rc.iterrows():
  subreddit = row['subreddit']
  comment = row['body']
  text = f"Create a rude comment about: {subreddit}. Comment: {comment}"
  rc_data.append(text)

import csv
# Specify the CSV file path
csv_file_path = "comment_data.csv"

# Write the list to the CSV file
with open(csv_file_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Value'])  # Write header
    for item in rc_data:
        csv_writer.writerow([item])

### II. Training the Model

In [ ]:
!pip install -U accelerate --quiet
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.5 MB/s eta 0:00:00


In [ ]:
# Mount your Google Drive to access the data
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/AI Camp/Captivating Cupcakes/data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/AI Camp/Captivating Cupcakes/data'
/content


In [ ]:
import csv

# Specify the CSV file path
csv_file_path = "comment_data.csv"

# Read the list from the CSV file
data = []
with open(csv_file_path, mode='r') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # Skip the header row
    for row in csv_reader:
        data.append(row[0])

FileNotFoundError: ignored

In [ ]:
import torch
from torch.utils.data import Dataset, random_split
from transformers import Trainer, TrainingArguments

# Custom dataset class to load dataset
class CommentDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            # Encode the descriptions using the GPT-Neo tokenizer
            encodings_dict = tokenizer('<|startoftext|>'
                                        + txt +
                                        '<|endoftext|>',
                                        truncation=True,
                                        max_length=max_length,
                                            padding="max_length")
            input_ids = torch.tensor(encodings_dict['input_ids'])
            self.input_ids.append(input_ids)
            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# Set the random seed to a fixed value to get reproducible results
#torch.manual_seed(42)

# Download the pre-trained GPT-Neo model's tokenizer
# Add the custom tokens denoting the beginning and the end
# of the sequence and a special token for padding
tokenizer = AutoTokenizer.from_pretrained("gpt2",
                            bos_token='<|startoftext|>',
                            eos_token='<|endoftext|>',
                            pad_token='<|pad|>')

# Download the pre-trained GPT-Neo model and transfer it to the GPU
model = AutoModelForCausalLM.from_pretrained("gpt2").cuda()

# Resize the token embeddings because we've just added 3 new tokens
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 50259. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(50259, 768)

In [ ]:
max_length = 200

# Load dataset
dataset = CommentDataset(data[0:20000], tokenizer, max_length)

# Split data into train/val
train_size = int(0.9 * len(dataset))

train_data, val_data = random_split(dataset, [train_size, len(dataset) - train_size])

max_length

200

In [ ]:
tokenizer.batch_decode(val_data[10])

['<|startoftext|>Create a rude comment about: FortNiteBR. Comment: *neither\n\nand that’s my point, epic. doesn’t. care. and if they do they haven’t been showing it, my point isn’t only about siphon, it’s an example. i’m more worried about the bugs like turbo building and audio that ruin the normal games we play. then there’s the huge argument about a FOV slider that i’m not going to get into as well.<|endoftext|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|

In [ ]:
MAX_EPOCHS = 10
for epoch in range(MAX_EPOCHS):
  training_args = TrainingArguments(output_dir=f'taro/comments/results',
                                      num_train_epochs=1,
                                      logging_steps=1000,
                                      save_steps=5000,
                                      evaluation_strategy='steps',
                                      eval_steps=1000,
                                      per_device_train_batch_size=2,
                                      per_device_eval_batch_size=2,
                                      warmup_steps=100,
                                      learning_rate=5e-5,
                                      weight_decay=0.01,
                                      logging_dir=f'taro/comments/logs')

  trainer = Trainer(model=model, args=training_args,
                    train_dataset=train_data,
                    eval_dataset=val_data,
                    # This custom collate function is necessary
                    # to built batches of data
                    data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                'attention_mask': torch.stack([f[1] for f in data]),
                'labels': torch.stack([f[0] for f in data])})

  # Start training process!
  trainer.train()

  # Save model in the specified file path
  trainer.save_model("taro/comments/model")
  tokenizer.save_pretrained("taro/comments/model")

In [ ]:
training_args = TrainingArguments(output_dir=f'taro/comments/results',
                                      num_train_epochs=2,
                                      logging_steps=5000,
                                      save_steps=5000,
                                      evaluation_strategy='steps',
                                      eval_steps=5000,
                                      per_device_train_batch_size=2,
                                      per_device_eval_batch_size=2,
                                      warmup_steps=100,
                                      learning_rate=5e-5,
                                      weight_decay=0.01,
                                      logging_dir=f'taro/comments/logs')

trainer = Trainer(model=model, args=training_args,
                  train_dataset=train_data,
                  eval_dataset=val_data,
                  # This custom collate function is necessary
                  # to built batches of data
                  data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
              'attention_mask': torch.stack([f[1] for f in data]),
              'labels': torch.stack([f[0] for f in data])})

# Start training process!
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
# Save model in the specified file path
trainer.save_model("taro/comments/model")
tokenizer.save_pretrained("taro/comments/model")

('taro/comments/model/tokenizer_config.json',
 'taro/comments/model/special_tokens_map.json',
 'taro/comments/model/vocab.json',
 'taro/comments/model/merges.txt',
 'taro/comments/model/added_tokens.json',
 'taro/comments/model/tokenizer.json')

In [ ]:
# Load model back up
tokenizer = AutoTokenizer.from_pretrained("taro/comments/model",
                            bos_token='<|startoftext|>',
                            eos_token='<|endoftext|>',
                            pad_token='<|pad|>')

# Download the pre-trained GPT-Neo model and transfer it to the GPU
model = AutoModelForCausalLM.from_pretrained("taro/model").cuda()

In [ ]:
!pip install huggingface_hub --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.3 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

In [ ]:
# Create your repo first to upload the model
api.create_repo(repo_id="commentbot")

RepoUrl('https://huggingface.co/taroii/commentbot', endpoint='https://huggingface.co', repo_type='model', repo_id='taroii/commentbot')

In [ ]:
%pwd

'/content/drive/MyDrive/AI Camp/Captivating Cupcakes/data'

In [ ]:
# Upload your model to huggingface. You can clone the repo anytime to use the model.
import os

model_pth = "./taro/comments/model"

files = os.listdir(model_pth)
for fi in files:
    print(os.path.join(model_pth, fi))

    api.upload_file(
        path_or_fileobj=os.path.join(model_pth, fi),
        path_in_repo=fi,
        repo_id="taroii/commentbot",
        repo_type="model",
    )

./taro/comments/model/tokenizer_config.json
./taro/comments/model/special_tokens_map.json
./taro/comments/model/added_tokens.json
./taro/comments/model/vocab.json
./taro/comments/model/merges.txt
./taro/comments/model/tokenizer.json
